In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
import random

# --- CONFIGURATION ---
MODELS = [
    {
        "name": "SmolLM2-135M",
        "repo": "HuggingFaceTB/SmolLM2-135M",
        "trust_remote_code": False
    },
    {
        "name": "RWKV7-0.1B",
        "repo": "fla-hub/rwkv7-0.1B-g1", 
        "trust_remote_code": True # Required for custom RWKV7 architecture
    }
]

# --- DATASET GENERATOR (100 Trials) ---
def generate_grammar_pairs(num_pairs=100):
    """
    Generates synthetic Subject-Verb agreement tests.
    Correct: "The cat runs."
    Wrong:   "The cat run."
    """
    singular_subjects = ["The cat", "A dog", "My friend", "The car", "He", "She", "The bird", "A computer", "The sun", "That man"]
    plural_subjects = ["The cats", "Dogs", "My friends", "The cars", "They", "We", "The birds", "Computers", "The stars", "Those men"]
    
    singular_verbs = ["runs", "eats", "jumps", "sleeps", "works", "shines", "moves", "flies", "falls", "plays"]
    plural_verbs =   ["run", "eat", "jump", "sleep", "work", "shine", "move", "fly", "fall", "play"]
    
    objects = ["fast", "food", "high", "well", "hard", "brightly", "quickly", "away", "down", "games"]

    pairs = []
    for _ in range(num_pairs):
        # 50% chance of generating a Singular Subject case
        if random.random() > 0.5:
            subj_idx = random.randint(0, len(singular_subjects)-1)
            verb_idx = random.randint(0, len(singular_verbs)-1)
            obj_idx = random.randint(0, len(objects)-1)
            
            good = f"{singular_subjects[subj_idx]} {singular_verbs[verb_idx]} {objects[obj_idx]}."
            bad =  f"{singular_subjects[subj_idx]} {plural_verbs[verb_idx]} {objects[obj_idx]}."
        else:
            # Plural Subject case
            subj_idx = random.randint(0, len(plural_subjects)-1)
            verb_idx = random.randint(0, len(plural_verbs)-1)
            obj_idx = random.randint(0, len(objects)-1)
            
            good = f"{plural_subjects[subj_idx]} {plural_verbs[verb_idx]} {objects[obj_idx]}."
            bad =  f"{plural_subjects[subj_idx]} {singular_verbs[verb_idx]} {objects[obj_idx]}."
            
        pairs.append((good, bad))
    
    return pairs

# --- SCORING ENGINE ---
def get_log_prob(model, tokenizer, sentence):
    """
    Computes the total log probability of a sentence.
    """
    inputs = tokenizer(sentence, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")
        
    input_ids = inputs.input_ids
    
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
    
    # Shift logits and labels for next-token prediction
    # logits[0, :-1] predicts input_ids[0, 1:]
    shift_logits = logits[0, :-1, :]
    shift_labels = input_ids[0, 1:]
    
    # Calculate Cross Entropy (Loss = -Avg Log Prob)
    # reduction='sum' gives total negative log prob
    loss = F.cross_entropy(shift_logits, shift_labels, reduction='sum')
    
    # Return Log Prob (negative of loss)
    return -loss.item()

# --- MAIN BATTLE LOOP ---
def run_battle():
    # 1. Generate Data
    print(f"Generating 100 grammar test pairs...")
    test_data = generate_grammar_pairs(100)
    print(f"Example Pair: Correct='{test_data[0][0]}' vs Wrong='{test_data[0][1]}'\n")

    results = {}

    # 2. Test Models
    for config in MODELS:
        name = config['name']
        print(f"--- Loading {name} ---")
        try:
            tokenizer = AutoTokenizer.from_pretrained(config['repo'], trust_remote_code=config['trust_remote_code'])
            model = AutoModelForCausalLM.from_pretrained(
                config['repo'], 
                trust_remote_code=config['trust_remote_code'],
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
            )
            
            if torch.cuda.is_available():
                model = model.cuda()
            
            print(f"Running 100 trials on {name}...")
            correct_count = 0
            
            for good, bad in test_data:
                score_good = get_log_prob(model, tokenizer, good)
                score_bad = get_log_prob(model, tokenizer, bad)
                
                if score_good > score_bad:
                    correct_count += 1
            
            results[name] = correct_count
            print(f"Finished {name}. Cleaning up GPU...")
            
            # Cleanup
            del model
            del tokenizer
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"FAILED to load or run {name}: {e}")
            results[name] = "Error"

    # --- FINAL SCOREBOARD ---
    print(f"\n{'='*30}")
    print(f"FINAL SCOREBOARD (Out of 100)")
    print(f"{'='*30}")
    for name, score in results.items():
        print(f"{name}: {score} / 100")

if __name__ == "__main__":
    run_battle()

/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generating 100 grammar test pairs...
Example Pair: Correct='The cat runs away.' vs Wrong='The cat run away.'

--- Loading SmolLM2-135M ---


`torch_dtype` is deprecated! Use `dtype` instead!
Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


Running 100 trials on SmolLM2-135M...
Finished SmolLM2-135M. Cleaning up GPU...
--- Loading RWKV7-0.1B ---


/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/fla/layers/rwkv7.py:142: UserWarning: According to Bo, you are using a potentially buggy FLA implementation of RWKV. If you plan to report any numbers based on this implementation, we strongly recommend cross-checking with the official repo: https://github.com/BlinkDL/RWKV-LM. Bo may disagree with results reported from this version.
  warnings.warn(


Running 100 trials on RWKV7-0.1B...


/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/fla/ops/rwkv7/fused_recurrent.py:301: UserWarning: Input tensor shape suggests potential format mismatch: seq_len (5) < num_heads (12). This may indicate the inputs were passed in head-first format [B, H, T, ...] when head_first=False was specified. Please verify your input tensor format matches the expected shape [B, T, H, ...].
  warnings.warn(
/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/fla/ops/rwkv7/fused_recurrent.py:301: UserWarning: Input tensor shape suggests potential format mismatch: seq_len (5) < num_heads (12). This may indicate the inputs were passed in head-first format [B, H, T, ...] when head_first=False was specified. Please verify your input tensor format matches the expected shape [B, T, H, ...].
  warnings.warn(
/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/fla/ops/rwkv7/fused_recurrent.py:301: UserWarning: Input tensor shape suggests potential format mismatch: seq_len (4) < nu

Finished RWKV7-0.1B. Cleaning up GPU...

FINAL SCOREBOARD (Out of 100)
SmolLM2-135M: 94 / 100
RWKV7-0.1B: 95 / 100


In [2]:
import torch
import time
import numpy as np
from transformers import AutoModelForCausalLM

# --- CONFIGURATION ---
BATCH_SIZE = 512
SEQ_LEN = 4      # Simulating a 4-token word (e.g., "unbelievable")
NUM_ITERS = 100  # Number of trials for averaging
WARMUP = 10      # Warmup steps to settle GPU clocks

# Define Models
MODELS = {
    "SmolLM2-135M": "HuggingFaceTB/SmolLM2-135M",
    "RWKV7-0.1B": "fla-hub/rwkv7-0.1B-g1"
}

def benchmark_model(name, repo_id):
    print(f"\n--- Benchmarking {name} ---")
    try:
        # Load Model
        # trust_remote_code=True is essential for RWKV7 / FLA
        model = AutoModelForCausalLM.from_pretrained(
            repo_id, 
            trust_remote_code=True,
            torch_dtype=torch.float16
        ).cuda()
        model.eval()

        # Generate Random Inputs
        # Shape: [Batch, Seq_Len]
        input_ids = torch.randint(0, 1000, (BATCH_SIZE, SEQ_LEN)).cuda()

        # Warmup (Get GPU clocks up, compile kernels)
        print("Warming up...")
        for _ in range(WARMUP):
            with torch.no_grad():
                _ = model(input_ids)
        torch.cuda.synchronize()

        # Benchmark Loop
        print(f"Running {NUM_ITERS} iterations...")
        times = []
        
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)

        with torch.no_grad():
            for _ in range(NUM_ITERS):
                start_event.record()
                
                # The Critical Operation: Forward Pass
                # For RWKV, this runs in "Parallel/Flash" mode for the chunk
                _ = model(input_ids)
                
                end_event.record()
                torch.cuda.synchronize() # Wait for GPU to finish
                times.append(start_event.elapsed_time(end_event)) # Returns ms

        avg_time = np.mean(times)
        std_time = np.std(times)
        
        print(f"Results for {name}:")
        print(f"  Avg Time: {avg_time:.4f} ms")
        print(f"  Std Dev:  {std_time:.4f} ms")
        print(f"  Throughput: {(BATCH_SIZE * SEQ_LEN) / (avg_time / 1000):.2f} tokens/sec")

        # Cleanup to free VRAM for next model
        del model
        torch.cuda.empty_cache()
        
        return avg_time

    except Exception as e:
        print(f"Failed to benchmark {name}: {e}")
        return float('inf')

# --- MAIN ---
if __name__ == "__main__":
    results = {}
    for name, repo in MODELS.items():
        results[name] = benchmark_model(name, repo)

    print("\n" + "="*30)
    print("FINAL COMPARISON (Lower is Better)")
    print("="*30)
    
    baseline = results["SmolLM2-135M"]
    for name, t in results.items():
        diff = ((t - baseline) / baseline) * 100
        print(f"{name}: {t:.2f} ms ({diff:+.1f}%)")


--- Benchmarking SmolLM2-135M ---
Warming up...
Running 100 iterations...
Results for SmolLM2-135M:
  Avg Time: 9.9147 ms
  Std Dev:  0.0210 ms
  Throughput: 206563.01 tokens/sec

--- Benchmarking RWKV7-0.1B ---


/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/fla/layers/rwkv7.py:142: UserWarning: According to Bo, you are using a potentially buggy FLA implementation of RWKV. If you plan to report any numbers based on this implementation, we strongly recommend cross-checking with the official repo: https://github.com/BlinkDL/RWKV-LM. Bo may disagree with results reported from this version.
  warnings.warn(


Warming up...
Running 100 iterations...


/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/fla/ops/rwkv7/fused_recurrent.py:301: UserWarning: Input tensor shape suggests potential format mismatch: seq_len (4) < num_heads (12). This may indicate the inputs were passed in head-first format [B, H, T, ...] when head_first=False was specified. Please verify your input tensor format matches the expected shape [B, T, H, ...].
  warnings.warn(


Results for RWKV7-0.1B:
  Avg Time: 7.8858 ms
  Std Dev:  0.3649 ms
  Throughput: 259708.59 tokens/sec

FINAL COMPARISON (Lower is Better)
SmolLM2-135M: 9.91 ms (+0.0%)
RWKV7-0.1B: 7.89 ms (-20.5%)


In [1]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer

# --- CONFIGURATION ---
MODEL_ID = "fla-hub/rwkv7-0.1B-g1"
BATCH_SIZE = 2000  # The massive number of beams
SEQ_LEN = 4        # The chunk size (e.g., scoring a 4-token word)
DTYPE = torch.float16
DEVICE = "cuda"

def get_mem_mb():
    """Returns current and peak memory usage in MB"""
    current = torch.cuda.memory_allocated() / 1024**2
    peak = torch.cuda.max_memory_allocated() / 1024**2
    return current, peak

def reset_peak():
    torch.cuda.reset_peak_memory_stats()

print(f"{'='*40}")
print(f"RWKV-7 Memory Profiler")
print(f"Config: Batch={BATCH_SIZE} | Seq={SEQ_LEN} | Dtype={DTYPE}")
print(f"{'='*40}\n")

# 1. BASELINE (Empty GPU)
torch.cuda.empty_cache()
reset_peak()
curr, peak = get_mem_mb()
print(f"1. Empty GPU:         {curr:.1f} MB")

# 2. LOAD MODEL (Weights)
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, 
    trust_remote_code=True, 
    torch_dtype=DTYPE
).to(DEVICE)

# Run one dummy tiny step to initialize internal buffers
model(torch.tensor([[0]], device=DEVICE)) 
torch.cuda.synchronize()
reset_peak()

curr, peak = get_mem_mb()
model_size = curr
print(f"2. Model Weights:     {curr:.1f} MB (Static Cost)")

# 3. CREATE GIANT STATE (The 'Cache')
# We simulate your exact setup: 2000 beams initialized from <BOS>
print("\nAllocating Giant State for 2000 beams...")

# Get shape from a single pass
with torch.no_grad():
    out = model(torch.tensor([[tokenizer.bos_token_id]], device=DEVICE), use_cache=True)
    seed_state = out.past_key_values # Shape: [Layers, 1, ...]

# Expand to 2000 beams
# RWKV states are Tensors, so we repeat them.
# We detach to ensure it's treated as leaf storage, not part of a graph
giant_state = seed_state.repeat(1, BATCH_SIZE, 1, 1).detach().clone()

torch.cuda.synchronize()
curr, peak = get_mem_mb()
state_size = curr - model_size
print(f"3. State Cache:       {state_size:.1f} MB (Persistent Cost)")
print(f"   (This stays CONSTANT even if sequence length increases!)")

# 4. FORWARD PASS (Chunked Mode)
# Simulating scoring a 4-token word for 2000 beams
print(f"\nRunning Forward Pass (Batch={BATCH_SIZE}, Seq={SEQ_LEN})...")
input_ids = torch.randint(0, 1000, (BATCH_SIZE, SEQ_LEN), device=DEVICE)

reset_peak() # Reset peak to capture ONLY this forward pass spike

with torch.no_grad():
    # We pass the giant state as 'past_key_values'
    outputs = model(input_ids, past_key_values=giant_state, use_cache=True)
    
    # Force realization of logits and new states
    logits = outputs.logits
    new_states = outputs.past_key_values
    
torch.cuda.synchronize()
_, run_peak = get_mem_mb()
activation_cost = run_peak - (model_size + state_size)

print(f"4. Forward Spike:     {activation_cost:.1f} MB (Temporary Cost)")

# --- SUMMARY ---
print(f"\n{'='*40}")
print(f"FINAL REPORT for Batch={BATCH_SIZE}")
print(f"{'='*40}")
print(f"Model Weights:      {model_size:>7.1f} MB")
print(f"KV/State Cache:     {state_size:>7.1f} MB")
print(f"Activation Spike:   {activation_cost:>7.1f} MB")
print(f"{'-'*40}")
print(f"TOTAL VRAM REQUIRED: {run_peak:>7.1f} MB")
print(f"{'='*40}")

# Warning about Logits
vocab_size = model.config.vocab_size
logit_size_mb = (BATCH_SIZE * SEQ_LEN * vocab_size * 2) / 1024**2
print(f"\nNOTE: The 'Activation Spike' is dominated by the Logits tensor.")
print(f"Logits Shape: [{BATCH_SIZE}, {SEQ_LEN}, {vocab_size}] (FP16)")
print(f"Theoretical Logit Size: ~{logit_size_mb:.1f} MB")

/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RWKV-7 Memory Profiler
Config: Batch=2000 | Seq=4 | Dtype=torch.float16

1. Empty GPU:         0.0 MB
Loading model...


Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
`torch_dtype` is deprecated! Use `dtype` instead!
/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/fla/layers/rwkv7.py:142: UserWarning: According to Bo, you are using a potentially buggy FLA implementation of RWKV. If you plan to report any numbers based on this implementation, we strongly recommend cross-checking with the official repo: https://github.com/BlinkDL/RWKV-LM. Bo may disagree with results reported from this version.
  warnings.warn(
/home/ebrahim/brainaudio/.venv/lib/python3.12/site-packages/fla/ops/rwkv7/fused_recurrent.py:301: UserWarning: Input tensor shape suggests potential format mismatch: seq_len (1) < num_heads (12). This may indicate the inputs were passed in head-first format [B, H, T, ...] when head_first=False was specified. Please verify your input tensor format mat

2. Model Weights:     379.5 MB (Static Cost)

Allocating Giant State for 2000 beams...


AttributeError: 'Cache' object has no attribute 'repeat'